In [1]:
# Load the .env file
#pip install -U python-dotenv
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# Cohere RAG pipeline

## Cohere container's backend

In [1]:
import requests
import cohere
import os
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')

# Define the API endpoint for streaming
url = "http://localhost:8000/v1/chat"
bearer = os.getenv('BEARER_SECRET_KEY')

# Set headers
headers = {
    "User-Id": "me",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {bearer}",
    "Cohere-Stream": "true",  # Enable streaming for chatbot responses
}

# Set the message to send
message = "What is Tech Innovators Inc.'s approach to workplace harassment?"

# Create the payload as a JSON dictionary
data = {"message": message}

# Send the POST request using requests
response = requests.post(url, headers=headers, json=data)

# Check for successful response
if response.status_code == 200:
  # Handle streaming response
  for line in response.iter_lines():
    # Decode the response (if necessary)
    decoded_line = line.decode("utf-8")
    # Process the received data from the stream (print it here)
    print(decoded_line)
else:
  print(f"Error: {response.status_code}")

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


{"response_id":"e973535a-6f5f-4396-a010-82285e7d5379","generation_id":"9c1e4f60-41a5-405d-9147-0ba61de3ea56","chat_history":[{"role":"USER","message":"What is Tech Innovators Inc.'s approach to workplace harassment?","tool_plan":null,"tool_results":null,"tool_calls":null},{"role":"CHATBOT","message":"Tech Innovators Inc. has a zero-tolerance policy towards any form of workplace harassment. We strive to foster a safe, respectful, and inclusive work environment, and any instance of harassment is taken very seriously. \n\nOur approach involves a detailed anti-harassment policy outlined in our employee handbook, which is accessible to all employees. This policy clearly defines what constitutes harassment, providing examples of different forms of harassment, including sexual harassment, and the consequences of such behavior. \n\nWe regularly conduct mandatory training sessions to educate employees on workplace harassment, ensuring everyone understands their rights and responsibilities. Thes

## Simple RAG pipeline (For test purpose only!)

This RAG pipeline was created to test LangSmith evaluation tools. Do not use if you are able to access Cohere toolkit backend.

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI()
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [4]:
# Define MongoDB vector database
os.environ["ATLAS_CONNECTION_STRING"] = os.getenv("ATLAS_CONNECTION_STRING")
client = MongoClient(os.environ["ATLAS_CONNECTION_STRING"])
db_name = "tech_innovators_db"
collection_name = "tech_innovators_collection"
atlas_collection = client[db_name][collection_name]
index_name = "vector_index_erp"

In [5]:
# Define vector store and retriever
def get_vector_store_retriver(index_name, embedding_model, collection):

  vector_store = MongoDBAtlasVectorSearch(
      embedding = embedding_model,
      collection = atlas_collection,
      index_name = index_name
  )

  retriever = vector_store.as_retriever(
      search_type = "similarity",
      search_kwargs = { "k": 10 }
  )

  return(vector_store, retriever)

vector_store, retriever = get_vector_store_retriver("vector_index_erp", embedding_model, atlas_collection)

In [6]:
# get one example question from the dataset in LangSmith for testing
from langsmith import Client

client = Client()
examples = list(client.list_examples(dataset_name="hr test"))

q = examples[0].inputs
q

{'question': "What information should be included in the policies and procedures section of the company's welcome message?"}

In [7]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain import hub

In [8]:
# Retrieve and generate using the relevant snippets from the docs
# Instantiate Atlas Vector Search as a retriever
vectorstore_retriever  = vector_store.as_retriever(
   search_type = "similarity",
   search_kwargs = { "k": 10 }
)

# load a RAG prompt from Langchain HUB
prompt = hub.pull("rlm/rag-prompt")
# our llm of choice
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def ragas_output_parser(docs):
    return [doc.page_content for doc in docs]

d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langsmith\client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [9]:
from langchain_core.runnables import RunnableParallel

generator = prompt | llm | StrOutputParser()

retriever = RunnableParallel(
    {
        "context": vectorstore_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
)

filter_langsmith_dataset = RunnableLambda(
    lambda x: x["question"] if isinstance(x, dict) else x
)

rag_chain = RunnableParallel(
    {
        "question": filter_langsmith_dataset,
        "answer": filter_langsmith_dataset | retriever | generator,
        "contexts": filter_langsmith_dataset
        | vectorstore_retriever
        | ragas_output_parser,
    }
)

In [10]:
# check with the example question to see if everything is working
get_answer = RunnableLambda(lambda x: x["answer"])
resp = (rag_chain | get_answer).invoke(q)
resp

"The policies and procedures section of the company's welcome message should include information on employee conduct, dress code, attendance expectations, and any other important guidelines for working at the company. It should also cover topics such as safety protocols, data security measures, and communication channels within the organization. Additionally, the policies and procedures section should outline the process for reporting any issues or concerns to the appropriate personnel."

## Open a test dataset in LangSmith

In [11]:
# This cell open a test dataset from LangSmith
# It will check whether the dataset name exist in LangSmith.
# If there is a dataset with that name, it will load the dataset. If not, it will generate an error message
from langsmith import Client
from langsmith.utils import LangSmithError

# Create a LangSmith client object
client = Client()
dataset_name = "hr test"

try:
    # check if dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("using existing dataset: ", dataset.name)
except LangSmithError:
    print("No dataset exist: ", dataset.name)

using existing dataset:  hr test


## Run evaluation on LangSmith

### Baseline (Without RAG)

Before evaluating the RAG pipeline, 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

Question: {question}

Helpful Answer:"""
llm_prompt = PromptTemplate.from_template(template)

just_llm = (
    {"question": RunnablePassthrough()}
    | llm_prompt
    | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    | StrOutputParser()
    | RunnableParallel(
        {
            "answer": RunnablePassthrough(),
            "contexts": RunnableLambda(lambda _: [""]),
        }
    )
)

In [ ]:
from ragas.integrations.langchain import EvaluatorChain
# the metric we will be using
from ragas.metrics import answer_correctness
from ragas.integrations.langsmith import evaluate

# evaluate just llm
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=just_llm,
    experiment_name="just_llm_1",
    metrics=[answer_correctness],
    verbose=True,
)

### With RAG

In [18]:
from ragas.integrations.langchain import EvaluatorChain
# the metric we will be using
from ragas.metrics import answer_correctness
from ragas.integrations.langsmith import evaluate
#from ragas import evaluate

In [20]:
dataset_name = "hr test"
# evaluate rag_chain
run = evaluate(
    dataset_name=dataset_name,
    llm_or_chain_factory=rag_chain,
    experiment_name="rag_chain_5",
    #metrics=[answer_correctness],
    verbose=True,
)

View the evaluation results for project 'rag_chain_5' at:
https://smith.langchain.com/o/08bc9556-81b3-56d7-98aa-4f87d6cdfca5/datasets/f04f14f3-f165-48c3-8d94-dbf759844c7d/compare?selectedSessions=da581b53-3f5c-4dac-a714-f328ea6f9693

View all tests for Dataset hr test at:
https://smith.langchain.com/o/08bc9556-81b3-56d7-98aa-4f87d6cdfca5/datasets/f04f14f3-f165-48c3-8d94-dbf759844c7d
[>                                                 ] 0/9

Error evaluating run bbf5e936-5fea-436f-89e3-3e217e5a5b1d with EvaluatorChain: RuntimeError("There is no current event loop in thread 'ThreadPoolExecutor-40_1'.")
Traceback (most recent call last):
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain_core\tracers\evaluation.py", line 127, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
                        ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\ragas\integrations\langchain.py", line 210, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 164, in invoke
    raise e
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 154, in invoke
    self._call(inputs, run_manager=run_manager)
  File "d:\Document\GitHu

[----->                                            ] 1/9

Error evaluating run 8f3c4545-8bcf-414b-9b0b-12cfa58a6d16 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'ThreadPoolExecutor-40_3'.")
Traceback (most recent call last):
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain_core\tracers\evaluation.py", line 127, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
                        ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\ragas\integrations\langchain.py", line 210, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 164, in invoke
    raise e
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 154, in invoke
    self._call(inputs, run_manager=run_manager)
  File "d:\Document\GitHu

[---------->                                       ] 2/9

...

Error evaluating run 27e06bc8-ac04-47ac-bf05-31e9a4848db6 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'ThreadPoolExecutor-40_0'.")
Traceback (most recent call last):
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain_core\tracers\evaluation.py", line 127, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
                        ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\ragas\integrations\langchain.py", line 210, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 164, in invoke
    raise e
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 154, in invoke
    self._call(inputs, run_manager=run_manager)
  File "d:\Document\GitHu

[------------------------------------------->      ] 8/9

Error evaluating run b30a0e8d-1712-429d-9f54-e3b36face358 with EvaluatorChain: RuntimeError("There is no current event loop in thread 'ThreadPoolExecutor-40_3'.")
Traceback (most recent call last):
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain_core\tracers\evaluation.py", line 127, in _evaluate_in_project
    evaluation_result = evaluator.evaluate_run(
                        ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\ragas\integrations\langchain.py", line 210, in evaluate_run
    eval_output = self.invoke(chain_eval, include_run_info=True)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 164, in invoke
    raise e
  File "d:\Document\GitHub\cohere-rag-eval\venv\Lib\site-packages\langchain\chains\base.py", line 154, in invoke
    self._call(inputs, run_manager=run_manager)
  File "d:\Document\GitHu

[------------------------------------------------->] 9/9

,error,execution_time,run_id
count,0,9.000000,9
unique,0,NaN,9
top,NaN,NaN,4c8eb819-b5e6-4562-a7c3-32500687d470
freq,NaN,NaN,1
mean,NaN,1.931884,NaN
std,NaN,0.455481,NaN
min,NaN,1.218915,NaN
25%,NaN,1.609093,NaN
50%,NaN,1.936864,NaN
75%,NaN,2.215408,NaN
